In [ ]:
# 引用必要模組
import numpy as np
import pandas as pd
from tensorflow import keras

In [ ]:
# 載入寶可夢資料
pokemon_df = pd.read_csv("pokemon.csv")
pokemon_df

In [ ]:
pokemon_df = pokemon_df.set_index("#")

In [ ]:
pokemon_df["Type 2"].fillna("empty", inplace=True)
pokemon_df["Type 1"] = pokemon_df["Type 1"].astype("category")
pokemon_df["Type 2"] = pokemon_df["Type 2"].astype("category")
pokemon_df["Legendary"] = pokemon_df["Legendary"].astype("int")

df_type1_onehot = pd.get_dummies(pokemon_df["Type 1"])
df_type2_onehot = pd.get_dummies(pokemon_df["Type 2"])
combine_df_onehot = df_type1_onehot.add(df_type2_onehot, fill_value=0).astype("int")

pd.options.display.max_columns = 30
pokemon_df = pokemon_df.join(combine_df_onehot)

mean = pokemon_df.loc[:, "HP":"Generation"].mean()
std = pokemon_df.loc[:, "HP":"Generation"].std()
pokemon_df.loc[:, "HP":"Generation"] = (pokemon_df.loc[:, "HP":"Generation"] - mean) / std

# print(pokemon_df.dtypes)
# pokemon_df.head()

In [ ]:
# 載入模型
model_2 = keras.models.load_model("logs/models/best-model-2.h5")

In [ ]:
# 隨機抽樣兩個數字(0~799)
two_pokemon = np.random.choice(np.arange(800), size=2, replace=True)

first_pokemon = two_pokemon[0]+1
second_pokemon = two_pokemon[1]+1

first_pokemon_name = pokemon_df.loc[first_pokemon, "Name"]
second_pokemon_name = pokemon_df.loc[second_pokemon, "Name"]

first_pokemon_data = np.array(pokemon_df.loc[first_pokemon, "HP":]).astype("float64")
second_pokemon_data = np.array(pokemon_df.loc[second_pokemon, "HP":]).astype("float64")

first_pokemon_data = np.expand_dims(first_pokemon_data, axis=0)
second_pokemon_data = np.expand_dims(second_pokemon_data, axis=0)

In [ ]:
# 預測兩隻寶可夢的對戰勝率
pred = model_2.predict(np.concatenate([first_pokemon_data, second_pokemon_data], axis=-1))
winner = first_pokemon_name if pred < 0.5 else second_pokemon_name

# 印出對戰結果
print(f"{first_pokemon_name}({first_pokemon}) vs. {second_pokemon_name}({second_pokemon})")
print("pred={:.6f}, winner is {}!".format(pred[0][0], winner))